Q learning cabs project source code

In [1]:
pip install gym[toy_text]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 59.6 MB/s eta 0:00:00
  Attempting uninstall: pygame
    Found existing installation: pygame 2.5.1
    Uninstalling pygame-2.5.1:
      Successfully uninstalled pygame-2.5.1


In [2]:
import gym

In [3]:
env = gym.make("Taxi-v3").env
new_step_api=True

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [4]:
a = env.action_space.n
print(a)

6


In [5]:
env.observation_space.n

500

In [7]:
env.reset()

162

In [8]:
state=env.encode(3,1,2,0)
state

328

In [9]:
env.s=state

In [10]:
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

Solving without RL

In [11]:
env.s=328
steps=0
penalties,reward=0,0

frames=[]

done=False

while not done:
  action=env.action_space.sample()
  state,reward,done,info =env.step(action)

  if reward == -10:
    penalties+=1

  #put each rendered frame into dict for animation

  frames.append({'frame': env.render(mode='ansi'),
                 'state':state,
                 'action':action,
                 'reward':reward})

  steps+=1


print(steps)
print(penalties)

1521
485


/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


In [22]:
#Exploring the environment to get the optimised path
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
  for i, frame in enumerate(frames):
    clear_output(wait=True)
    print(frame['frame'])
    print(f"Timestep:{i+1}")
    print(f"State:{frame['state']}")
    print(f"Action:{frame['action']}")
    print(f"Reward:{frame['reward']}")
    sleep(.1)

print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep:1521
State:410
Action:5
Reward:20


Solving with RL

In [13]:
import numpy as np
q_table = np.zeros([env.observation_space.n,env.action_space.n])

In [14]:
#Training Agent

import random
from IPython.display import clear_output

#Hyper parameters

alpha=0.1
gamma=0.6
epsilon=0.1

#For plotting metrics

all_epochs=[]
all_penalties=[]

for i in range(1,100001):
  state=env.reset()

  epochs,penalties, reward = 0,0,0
  done=False

  while not done:
    if random.uniform(0,1) < epsilon:
      action= env.action_space.sample()
    else:
      action = np.argmax(q_table[state])

    next_state,reward,done,info = env.step(action)

    old_value= q_table[state,action]

    next_max=np.max(q_table[next_state])

    new_value = (1-alpha) * old_value + \
                 alpha *(reward +gamma * next_max)

    q_table[state,action] = new_value

    if reward == -10:
      penalties+=1

    state= next_state
    epochs+=1

  if i%100 ==0:
    clear_output(wait=True)
    print(f"Episode:{i}")

print("Training Finished")

Episode:100000
Training Finished


In [15]:
q_table[328]

array([ -2.40855313,  -2.27325184,  -2.40357022,  -2.35686136,
       -10.08875108, -10.40283696])

In [16]:
#Evaluate agents performance after q-learning

total_epochs, total_penalties = 0,0
episodes = 10

frames1 = []

for episode in range(episodes):
  state=env.reset()
  epochs,penalties,reward =0,0,0
  done =False

  while not done:
    action=np.argmax(q_table[state])
    state,reward,done,info = env.step(action)

    if reward == -10:
      penalties+=1

    epochs+=1

    total_penalties +=penalties
    total_epochs +=epochs

    if episode == 1 :
        frames1.append({'frame': env.render(mode='ansi'),'state':state,'action':action,'reward':reward})

print(f"Results after {episodes} episodes:")
print(f"Average timestpes per episode:{total_epochs/episodes}")
print(f"Average penalties per episode:{total_penalties/episodes}")


Results after 10 episodes:
Average timestpes per episode:81.9
Average penalties per episode:0.0


In [21]:
print_frames(frames1)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep:12
State:410
Action:5
Reward:20
